# 원본

In [1]:
# Settings
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import time
import os

In [2]:
# 역대 박스오피스(~100) 불러오기
data = pd.read_excel('KOBIS_역대_박스오피스(통합전산망_집계_기준)_2022-12-15.xlsx', header=4)
data = data[:100]
data['개봉일'] = data['개봉일'].astype('datetime64')
m_year = data['개봉일'].dt.year.astype('string').tolist()
m_year[1] = '2018'      # 네이버가 극한직업 년도 잘못 설정함
m_year[9] = '2012'      # 네이버가 7번방의 선물 년도 잘못 설정함
m_year[14] = '2017'     # 네이버가 신과함께-인과 연 년도 잘못 설정함
m_year[25] = '2013'     # 네이버가 겨울왕국 년도 잘못 설정함
m_year[27] = '2015'     # 네이버가 검사외전 년도 잘못 설정함
m_year[37] = '2021'     # 네이버가 탑건: 매버릭 년도 잘못 설정함
m_year[41] = '2016'     # 네이버가 공조 년도 잘못 설정함
m_year[51] = '2021'     # 네이버가 한산: 용의 출현 년도 잘못 설정함
m_year[54] = '2012'     # 네이버가 베를린 년도 잘못 설정함
m_year[60] = '2021'     # 네이버가 공조2: 인터내셔날 년도 잘못 설정함
m_year[61] = '2015'     # 네이버가 럭키 년도 잘못 설정함
m_year[72] = '2014'     # 네이버가 사도 년도 잘못 설정함
m_year[94] = '2017'     # 네이버가 안시성 년도 잘못 설정함
m_year[98] = '2016'     # 네이버가 더 킹 년도 잘못 설정함
m_list = data['영화명'].astype('string').tolist()
data['링크연도'] = m_year
data.to_csv('KOBIS_TOP100_daum.csv')



In [3]:
data = pd.read_csv('KOBIS_TOP100_daum.csv')
m_list = data['영화명'].astype('string')
m_year = data['링크연도'].astype('string')

m_year[6] = '2014'      # 다음에서 베테랑 
m_year[14] = '2018'     # 다음에서 신과함께 - 인과연
m_year[34] = '2013'     # 다음에서 수상한 그녀
m_year[74] = '2014'     # 다음에서 킹스맨:시크릿 에이전트

In [4]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 나래 수정

In [5]:
# 평점 더 보기 클릭 
def review_more_click(more_num):
    more_cnt = 0
    
    while True:
        driver.find_element(By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_box > div.alex_more > button').click()
        
        time.sleep(0.5)
        
        more_cnt += 1
        if more_cnt == more_num:
            break

In [6]:
m_list[74], m_year[74]

('킹스맨 : 시크릿 에이전트', '2014')

In [7]:
# 다음 영화 사이트 열기
driver.get('https://movie.daum.net/main/')

review_num = 500
more_num = 50
review_lst = []
movie_rank = 0
is_first = 1

for movie, year in zip(m_list[10:74], m_year[10:74]):
    # 검색어 입력
    print(f'--- 영화 {movie}({year})를 검색합니다.', end=' ')
    
    if is_first == 1:
        input_box = driver.find_element(By.CSS_SELECTOR, 'input.tf_keyword')
    else: 
        driver.find_element(By.CLASS_NAME, 'link_search').send_keys(Keys.ENTER)
        input_box = driver.find_element(By.CSS_SELECTOR, 'body > div.kakao_wrap.detail_type > header > div.kakaohead_menu > div > div > form > fieldset > div > input')
    
    input_box.send_keys(movie)            # 입력
    input_box.send_keys(Keys.ENTER)         # Enter
    # input_box.clear()                     # 입력된 내용 지우기
    time.sleep(1)
    
    # 해당 영화 사이트로 이동하기
    driver.find_element(By.CLASS_NAME, 'movie').click()             # 영화 탭 클릭
    time.sleep(1)
    
    search_lst = driver.find_elements(By.CSS_SELECTOR, '#mainContent > div > div.box_searchinfo > div.detail_searchinfo.movie.search_result > ul > li')
    keyword = re.sub('[^0-9가-힣]', '', str(movie))       # 영화명 공백 및 특수문자 제거
    for search in search_lst:
        title_link = search.find_element(By.CSS_SELECTOR, 'div > div > strong > a')
        title = re.sub('[^0-9가-힣]', "", title_link.text.split('(')[0])
        # print(title)
        title_year = search.find_element(By.CSS_SELECTOR, 'div > div > dl > dd').text
        # print(title_year, type(title_year), year, type(year))
        if (title == keyword) & (title_year == year):
            # print([title, year])
            title_link.click()
            print(f'>>> 해당 페이지로 이동합니다.')
            break
    time.sleep(1)
    
    # 평점 탭 클릭
    sel = 'div.tabmenu_wrap > ul > li > a > span'
    driver.find_elements(By.CSS_SELECTOR, sel)[3].click()
    driver.find_elements(By.CSS_SELECTOR, sel)[3].click()
    time.sleep(1)

    # 정렬 '최신순' 선택하기
    sel2 = '#alex-area > div > div > div > div.cmt_box > ul.list_category > li:nth-child(3) > button'
    driver.find_element(By.CSS_SELECTOR, sel2).send_keys(Keys.ENTER) 
    time.sleep(1)
    
    # 평점 더보기 클릭하기
    try :
        review_more_click(more_num)
    except :
        pass

    # 한 페이지의 리뷰 전체 불러오기
    sel3 = '#alex-area > div > div > div > div.cmt_box > ul.list_comment > li'
    reviews = driver.find_elements(By.CSS_SELECTOR, sel3)

    review_cnt = 0
    print('Start!', end='')    
    for review in reviews:
        id = review.find_element(By.CSS_SELECTOR, 'div > strong > span > a').text
        
        # 중복인 게시글 건너뛰기
        if is_first == 0:
            if id == review_lst[-1][1]:
                continue
        else:
            pass
        
        # 평점 읽어오기 
        score = review.find_element(By.CSS_SELECTOR, 'div > div').text
        
        # 리뷰 읽어오기
        try:
            text = review.find_element(By.CSS_SELECTOR, 'div > p').text
            if len(text) == 1:          # 성의 없는 댓글(.)은 읽지 않기 
                continue
        except:
            continue
        
        # 데이터셋 만들기 
        review_lst.append([movie, text, score])
        review_cnt += 1
        
        # 메세지 출력
        if review_cnt % 10 == 0:
            if review_cnt != 10 and review_cnt % 100 == 10:
                print(f'\n> {review_cnt}', end=' ')
            else:
                print(f'> {review_cnt}', end=' ')
        
        if review_cnt == review_num:
            print('> END!')
            break
        
    # 영화 하나 끝남 메시지 출력
    is_first = 0
    movie_rank += 1
    print(f'박스오피스 랭킹 {movie_rank}위. {movie}의 리뷰 {review_cnt}개를 모두 수집했습니다.')
    print('-'*80)

--- 영화 알라딘(2019)를 검색합니다. >>> 해당 페이지로 이동합니다.
Start!> 10 > 20 > 30 > 40 > 50 > 60 > 70 > 80 > 90 > 100 
> 110 > 120 > 130 > 140 > 150 > 160 > 170 > 180 > 190 > 200 
> 210 > 220 > 230 > 240 > 250 > 260 > 270 > 280 > 290 > 300 
> 310 > 320 > 330 > 340 > 350 > 360 > 370 > 380 > 390 > 400 
> 410 > 420 > 430 > 440 > 450 > 460 > 470 > 480 > 490 > 500 > END!
박스오피스 랭킹 1위. 알라딘의 리뷰 500개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 암살(2015)를 검색합니다. >>> 해당 페이지로 이동합니다.
Start!> 10 > 20 > 30 > 40 > 50 > 60 > 70 > 80 > 90 > 100 
> 110 > 120 > 130 > 140 > 150 > 160 > 170 > 180 > 190 > 200 박스오피스 랭킹 2위. 암살의 리뷰 205개를 모두 수집했습니다.
--------------------------------------------------------------------------------
--- 영화 범죄도시2(2022)를 검색합니다. >>> 해당 페이지로 이동합니다.
Start!> 10 > 20 > 30 > 40 > 50 > 60 > 70 > 80 > 90 > 100 
> 110 > 120 > 130 > 140 > 150 > 160 > 170 > 180 > 190 > 200 
> 210 > 220 > 230 > 240 > 250 > 260 > 270 > 280 > 290 > 300 
> 310 > 320 > 330 > 340 

In [9]:
review_data = pd.DataFrame(review_lst, columns=['movie', 'review', 'score'])
review_data.to_csv('./review_data_daum10_74.csv', encoding='utf-8-sig')
review_data

,movie,review,score
0,알라딘,최고최고\n다시봐도 좋음\n노래도 배우도 굿,10
1,알라딘,음악도 연기도 캐스팅도 색채도 너무 다 좋아요\n언제봐도 몇번봐도 재미있는 영화,10
2,알라딘,"OST, 배우, CG, 스토리 등등 뭐 하나 나무랄데 없는 영화\n마지막 엔딩까지 ...",9
3,알라딘,이번에 재개봉해서 봤는데 영상도 잘만들었고 너무 재밌고 신나는 영화임 배우분...,10
4,알라딘,실사 영화덴 애니보다 나은,8
...,...,...,...
24667,아저씨,맷데이먼이. 충격좀 받겠다 이거\n나보다 잘치는데 하면서,10
24668,아저씨,원빈 연기 잘하네,9
24669,아저씨,액션영화로 참 시원하고 재밌게 본 영화 \n영화본후 세수할때 거울보고 오징어 목격한...,10
24670,아저씨,원빈... 신의 한 수,9


In [2]:
import pandas as pd
import numpy as np
import os
forders = os.listdir('daum')
print(forders)

['review_data_daum10.csv', 'review_data_daum10_74.csv', 'review_data_daum74_100.csv']


In [3]:
df_all = pd.DataFrame()
for i in range(0,len(forders)):
    if forders[i].split('.')[1] == 'csv':
        file = 'daum/'+forders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

In [5]:
df_all
df_all.to_csv('./review_data_daum.csv', encoding='utf-8-sig')

In [13]:
df_all = df_all[['movie', 'review', 'score']]

In [14]:
len(df_all)

40436

In [15]:
df_all

,movie,review,score
0,명량,정말 끝내주는 영화네요.한국영화 많이 보는 내가 이걸 인제 봤다니ㅎ,10
1,명량,한산보기전 다시 봤는데 역시 재미있네요!!,10
2,명량,명량\n실제로 거북선 불타지 않았구요 배설 해전 후에 죽었구요\n해전에서 백병전이다...,6
3,명량,명량해전 하나만 좋았다,6
4,명량,한산보고 궁금해서봤는데 명량이확실히 스토리도탄탄하고 전투씬도더박진감넘치고리얼하네요\...,8
...,...,...,...
11879,완득이,재미나게봤어요,10
11880,완득이,자연스런 전개 속에 다문화가정의 소시민적인 삶을 잘 담아냈당.ㅋ 유머도 너무재밌고 ...,10
11881,완득이,높은 평점때문에 엄청 수작일꺼라 생각했는데.. 기대치에 한참 못미쳤음; \n그래도 ...,7
11882,완득이,웃겨욬ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,10


In [16]:
df_all.reset_index(drop=False)

,index,movie,review,score
0,0,명량,정말 끝내주는 영화네요.한국영화 많이 보는 내가 이걸 인제 봤다니ㅎ,10
1,1,명량,한산보기전 다시 봤는데 역시 재미있네요!!,10
2,2,명량,명량\n실제로 거북선 불타지 않았구요 배설 해전 후에 죽었구요\n해전에서 백병전이다...,6
3,3,명량,명량해전 하나만 좋았다,6
4,4,명량,한산보고 궁금해서봤는데 명량이확실히 스토리도탄탄하고 전투씬도더박진감넘치고리얼하네요\...,8
...,...,...,...,...
40431,11879,완득이,재미나게봤어요,10
40432,11880,완득이,자연스런 전개 속에 다문화가정의 소시민적인 삶을 잘 담아냈당.ㅋ 유머도 너무재밌고 ...,10
40433,11881,완득이,높은 평점때문에 엄청 수작일꺼라 생각했는데.. 기대치에 한참 못미쳤음; \n그래도 ...,7
40434,11882,완득이,웃겨욬ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,10


In [19]:
df_all = df_all.reset_index(drop=True)
df_all

,movie,review,score
0,명량,정말 끝내주는 영화네요.한국영화 많이 보는 내가 이걸 인제 봤다니ㅎ,10
1,명량,한산보기전 다시 봤는데 역시 재미있네요!!,10
2,명량,명량\n실제로 거북선 불타지 않았구요 배설 해전 후에 죽었구요\n해전에서 백병전이다...,6
3,명량,명량해전 하나만 좋았다,6
4,명량,한산보고 궁금해서봤는데 명량이확실히 스토리도탄탄하고 전투씬도더박진감넘치고리얼하네요\...,8
...,...,...,...
40431,완득이,재미나게봤어요,10
40432,완득이,자연스런 전개 속에 다문화가정의 소시민적인 삶을 잘 담아냈당.ㅋ 유머도 너무재밌고 ...,10
40433,완득이,높은 평점때문에 엄청 수작일꺼라 생각했는데.. 기대치에 한참 못미쳤음; \n그래도 ...,7
40434,완득이,웃겨욬ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,10


In [20]:
df_all
df_all.to_csv('./review_data_daum40436.csv', encoding='utf-8-sig')